### Global imports

In [4]:
import os
from datetime import datetime
from math import floor
import copy
import torch
import wandb as wb
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import auc, f1_score, roc_curve
from packages.video_utils import H264Extractor, Video
from packages.constants import GOP_SIZE, FRAME_HEIGHT, FRAME_WIDTH, DATASET_ROOT, N_GOPS_FROM_DIFFERENT_DEVICE, N_GOPS_FROM_SAME_DEVICE, SAME_DEVICE_LABEL
from packages.dataset import VisionGOPDataset, GopPairDataset
from packages.common import create_custom_logger
from packages.network import H4vdmNet

### Initialize stuff

In [2]:
if not os.path.exists(DATASET_ROOT):
    raise Exception(f'Dataset root does not exist: {DATASET_ROOT}')

log = create_custom_logger('h4vdm.ipynb')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log.info(f'Using device: {device}')

h4vdm.ipynb - INFO - Using device: cuda


### Load GOP dataset

Remember to delete dataset.json if you want to add new devices/videos

In [3]:
bin_path = os.path.abspath(os.path.join(os.getcwd(), 'h264-extractor', 'bin'))
h264_ext_bin = os.path.join(bin_path, 'h264dec_ext_info')
h264_extractor = H264Extractor(bin_filename=h264_ext_bin, cache_dir=DATASET_ROOT)
Video.set_h264_extractor(h264_extractor)

vision_gop_dataset = VisionGOPDataset(
    root_path=DATASET_ROOT,
    devices=[],
    media_types = ['videos'],
    properties=[],
    extensions=['mp4', 'mov', '3gp'],
    gop_size=GOP_SIZE,
    frame_width=FRAME_WIDTH,
    frame_height=FRAME_HEIGHT,
    gops_per_video=4,
    build_on_init=False,
    force_rebuild=False,
    download_on_init=False,
    ignore_local_dataset=False,
    shuffle=False)

is_loaded = vision_gop_dataset.load()
if not is_loaded:
    log.info('Dataset was not loaded. Building...')
else:
    log.info('Dataset was loaded.')

print(f'Dataset length: {len(vision_gop_dataset)}')



h4vdm.ipynb - INFO - Dataset was loaded.


Dataset length: 1914


### Create training and testing datasets

In [4]:
devices = ['D01_Samsung_GalaxyS3Mini', 'D02_Apple_iPhone4s', 'D03_Huawei_P9', 'D04_LG_D290', 'D05_Apple_iPhone5c', 'D06_Apple_iPhone6', 'D07_Lenovo_P70A', 'D08_Samsung_GalaxyTab3', 'D09_Apple_iPhone4', 'D10_Apple_iPhone4s', 'D11_Samsung_GalaxyS3', 'D12_Sony_XperiaZ1Compact', 'D13_Apple_iPad2', 'D14_Apple_iPhone5c', 'D15_Apple_iPhone6', 'D16_Huawei_P9Lite', 'D17_Microsoft_Lumia640LTE', 'D18_Apple_iPhone5c', 'D19_Apple_iPhone6Plus', 'D20_Apple_iPadMini', 'D21_Wiko_Ridge4G', 'D22_Samsung_GalaxyTrendPlus', 'D23_Asus_Zenfone2Laser', 'D24_Xiaomi_RedmiNote3', 'D25_OnePlus_A3000', 'D26_Samsung_GalaxyS3Mini', 'D27_Samsung_GalaxyS5', 'D28_Huawei_P8', 'D29_Apple_iPhone5', 'D30_Huawei_Honor5c', 'D31_Samsung_GalaxyS4Mini', 'D32_OnePlus_A3003', 'D33_Huawei_Ascend', 'D34_Apple_iPhone5', 'D35_Samsung_GalaxyTabA']

print(f'{len(devices)} devices: {devices}')

training_set_1_devices = ['D25_OnePlus_A3000', 'D15_Apple_iPhone6', 'D19_Apple_iPhone6Plus', 'D10_Apple_iPhone4s', 'D07_Lenovo_P70A', 'D35_Samsung_GalaxyTabA', 'D01_Samsung_GalaxyS3Mini', 'D21_Wiko_Ridge4G', 'D29_Apple_iPhone5', 'D30_Huawei_Honor5c', 'D04_LG_D290', 'D20_Apple_iPadMini', 'D34_Apple_iPhone5', 'D03_Huawei_P9', 'D16_Huawei_P9Lite', 'D23_Asus_Zenfone2Laser', 'D31_Samsung_GalaxyS4Mini', 'D14_Apple_iPhone5c']
testing_set_1_devices = ['D18_Apple_iPhone5c', 'D13_Apple_iPad2', 'D28_Huawei_P8', 'D26_Samsung_GalaxyS3Mini', 'D02_Apple_iPhone4s', 'D32_OnePlus_A3003', 'D09_Apple_iPhone4', 'D24_Xiaomi_RedmiNote3', 'D22_Samsung_GalaxyTrendPlus', 'D05_Apple_iPhone5c', 'D27_Samsung_GalaxyS5', 'D06_Apple_iPhone6', 'D17_Microsoft_Lumia640LTE', 'D33_Huawei_Ascend', 'D11_Samsung_GalaxyS3', 'D08_Samsung_GalaxyTab3', 'D12_Sony_XperiaZ1Compact']

training_set_2_devices = ['D14_Apple_iPhone5c', 'D11_Samsung_GalaxyS3', 'D29_Apple_iPhone5', 'D01_Samsung_GalaxyS3Mini', 'D31_Samsung_GalaxyS4Mini', 'D17_Microsoft_Lumia640LTE', 'D25_OnePlus_A3000', 'D08_Samsung_GalaxyTab3', 'D26_Samsung_GalaxyS3Mini', 'D02_Apple_iPhone4s', 'D19_Apple_iPhone6Plus', 'D10_Apple_iPhone4s', 'D32_OnePlus_A3003', 'D07_Lenovo_P70A', 'D05_Apple_iPhone5c', 'D04_LG_D290', 'D28_Huawei_P8', 'D23_Asus_Zenfone2Laser']
testing_set_2_devices = ['D18_Apple_iPhone5c', 'D27_Samsung_GalaxyS5', 'D15_Apple_iPhone6', 'D12_Sony_XperiaZ1Compact', 'D30_Huawei_Honor5c', 'D33_Huawei_Ascend', 'D35_Samsung_GalaxyTabA', 'D34_Apple_iPhone5', 'D24_Xiaomi_RedmiNote3', 'D06_Apple_iPhone6', 'D22_Samsung_GalaxyTrendPlus', 'D20_Apple_iPadMini', 'D13_Apple_iPad2', 'D09_Apple_iPhone4', 'D03_Huawei_P9', 'D21_Wiko_Ridge4G', 'D16_Huawei_P9Lite']

training_set_3_devices = ['D20_Apple_iPadMini', 'D24_Xiaomi_RedmiNote3', 'D01_Samsung_GalaxyS3Mini', 'D09_Apple_iPhone4', 'D21_Wiko_Ridge4G', 'D11_Samsung_GalaxyS3', 'D33_Huawei_Ascend', 'D03_Huawei_P9', 'D13_Apple_iPad2', 'D10_Apple_iPhone4s', 'D06_Apple_iPhone6', 'D31_Samsung_GalaxyS4Mini', 'D34_Apple_iPhone5', 'D08_Samsung_GalaxyTab3', 'D07_Lenovo_P70A', 'D28_Huawei_P8', 'D23_Asus_Zenfone2Laser', 'D15_Apple_iPhone6']
testing_set_3_devices = ['D26_Samsung_GalaxyS3Mini', 'D32_OnePlus_A3003', 'D14_Apple_iPhone5c', 'D27_Samsung_GalaxyS5', 'D05_Apple_iPhone5c', 'D17_Microsoft_Lumia640LTE', 'D22_Samsung_GalaxyTrendPlus', 'D19_Apple_iPhone6Plus', 'D02_Apple_iPhone4s', 'D30_Huawei_Honor5c', 'D25_OnePlus_A3000', 'D12_Sony_XperiaZ1Compact', 'D04_LG_D290', 'D29_Apple_iPhone5', 'D18_Apple_iPhone5c', 'D35_Samsung_GalaxyTabA', 'D16_Huawei_P9Lite']

training_set_4_devices = ['D10_Apple_iPhone4s', 'D01_Samsung_GalaxyS3Mini', 'D04_LG_D290', 'D07_Lenovo_P70A', 'D29_Apple_iPhone5', 'D30_Huawei_Honor5c', 'D28_Huawei_P8', 'D08_Samsung_GalaxyTab3', 'D05_Apple_iPhone5c', 'D09_Apple_iPhone4', 'D21_Wiko_Ridge4G', 'D35_Samsung_GalaxyTabA', 'D20_Apple_iPadMini', 'D17_Microsoft_Lumia640LTE', 'D23_Asus_Zenfone2Laser', 'D14_Apple_iPhone5c', 'D19_Apple_iPhone6Plus', 'D18_Apple_iPhone5c']
testing_set_4_devices = ['D11_Samsung_GalaxyS3', 'D12_Sony_XperiaZ1Compact', 'D26_Samsung_GalaxyS3Mini', 'D15_Apple_iPhone6', 'D32_OnePlus_A3003', 'D03_Huawei_P9', 'D13_Apple_iPad2', 'D27_Samsung_GalaxyS5', 'D33_Huawei_Ascend', 'D22_Samsung_GalaxyTrendPlus', 'D34_Apple_iPhone5', 'D31_Samsung_GalaxyS4Mini', 'D25_OnePlus_A3000', 'D24_Xiaomi_RedmiNote3', 'D06_Apple_iPhone6', 'D16_Huawei_P9Lite', 'D02_Apple_iPhone4s']

training_set_5_devices = ['D35_Samsung_GalaxyTabA', 'D11_Samsung_GalaxyS3', 'D07_Lenovo_P70A', 'D22_Samsung_GalaxyTrendPlus', 'D17_Microsoft_Lumia640LTE', 'D33_Huawei_Ascend', 'D18_Apple_iPhone5c', 'D13_Apple_iPad2', 'D25_OnePlus_A3000', 'D09_Apple_iPhone4', 'D34_Apple_iPhone5', 'D24_Xiaomi_RedmiNote3', 'D03_Huawei_P9', 'D23_Asus_Zenfone2Laser', 'D20_Apple_iPadMini', 'D12_Sony_XperiaZ1Compact', 'D02_Apple_iPhone4s', 'D04_LG_D290', 'D26_Samsung_GalaxyS3Mini', 'D01_Samsung_GalaxyS3Mini', 'D27_Samsung_GalaxyS5', 'D06_Apple_iPhone6', 'D05_Apple_iPhone5c', 'D30_Huawei_Honor5c']
testing_set_5_devices = ['D14_Apple_iPhone5c', 'D15_Apple_iPhone6', 'D21_Wiko_Ridge4G', 'D19_Apple_iPhone6Plus', 'D16_Huawei_P9Lite', 'D08_Samsung_GalaxyTab3', 'D32_OnePlus_A3003', 'D29_Apple_iPhone5', 'D31_Samsung_GalaxyS4Mini', 'D28_Huawei_P8', 'D10_Apple_iPhone4s']

training_set_6_devices = ['D14_Apple_iPhone5c', 'D15_Apple_iPhone6', 'D21_Wiko_Ridge4G', 'D19_Apple_iPhone6Plus', 'D16_Huawei_P9Lite', 'D08_Samsung_GalaxyTab3', 'D32_OnePlus_A3003', 'D29_Apple_iPhone5', 'D31_Samsung_GalaxyS4Mini', 'D28_Huawei_P8', 'D10_Apple_iPhone4s', 'D24_Xiaomi_RedmiNote3', 'D03_Huawei_P9', 'D23_Asus_Zenfone2Laser', 'D20_Apple_iPadMini', 'D12_Sony_XperiaZ1Compact', 'D02_Apple_iPhone4s', 'D04_LG_D290', 'D26_Samsung_GalaxyS3Mini', 'D01_Samsung_GalaxyS3Mini', 'D27_Samsung_GalaxyS5', 'D06_Apple_iPhone6', 'D05_Apple_iPhone5c', 'D30_Huawei_Honor5c']
testing_set_6_devices = ['D35_Samsung_GalaxyTabA', 'D11_Samsung_GalaxyS3', 'D07_Lenovo_P70A', 'D22_Samsung_GalaxyTrendPlus', 'D17_Microsoft_Lumia640LTE', 'D33_Huawei_Ascend', 'D18_Apple_iPhone5c', 'D13_Apple_iPad2', 'D25_OnePlus_A3000', 'D09_Apple_iPhone4', 'D34_Apple_iPhone5']

training_set_7_devices = ['D14_Apple_iPhone5c', 'D15_Apple_iPhone6', 'D21_Wiko_Ridge4G', 'D19_Apple_iPhone6Plus', 'D16_Huawei_P9Lite', 'D08_Samsung_GalaxyTab3', 'D32_OnePlus_A3003', 'D29_Apple_iPhone5', 'D31_Samsung_GalaxyS4Mini', 'D28_Huawei_P8', 'D10_Apple_iPhone4s', 'D35_Samsung_GalaxyTabA', 'D11_Samsung_GalaxyS3', 'D07_Lenovo_P70A', 'D22_Samsung_GalaxyTrendPlus', 'D17_Microsoft_Lumia640LTE', 'D33_Huawei_Ascend', 'D18_Apple_iPhone5c', 'D13_Apple_iPad2', 'D25_OnePlus_A3000', 'D09_Apple_iPhone4', 'D34_Apple_iPhone5']
testing_set_7_devices = ['D24_Xiaomi_RedmiNote3', 'D03_Huawei_P9', 'D23_Asus_Zenfone2Laser', 'D20_Apple_iPadMini', 'D12_Sony_XperiaZ1Compact', 'D02_Apple_iPhone4s', 'D04_LG_D290', 'D26_Samsung_GalaxyS3Mini', 'D01_Samsung_GalaxyS3Mini', 'D27_Samsung_GalaxyS5', 'D06_Apple_iPhone6', 'D05_Apple_iPhone5c', 'D30_Huawei_Honor5c']

training_set_devices = [training_set_1_devices, training_set_2_devices, training_set_3_devices, training_set_4_devices, training_set_5_devices, training_set_6_devices, training_set_7_devices]
testing_set_devices = [testing_set_1_devices, testing_set_2_devices, testing_set_3_devices, testing_set_4_devices, testing_set_5_devices, testing_set_6_devices, testing_set_7_devices]

# training_set_devices = [training_set_1_devices]
# testing_set_devices = [testing_set_1_devices]

assert len(training_set_devices) == len(testing_set_devices), 'There must be the same number of training and testing sets'
n_datasets = len(training_set_devices)

for epoch in range(n_datasets):
    # print(f'Training set {epoch+1} devices: {training_set_devices[epoch]}')
    # print(f'Testing set {epoch+1} devices: {testing_set_devices[epoch]}')
    print(f'Training set {epoch+1} devices: ', end='')
    for name in training_set_devices[epoch]:
        print(f'{name[1:3]}, ', end='')
    print('')
    print(f'Testing set {epoch+1} devices: ', end='')
    for name in testing_set_devices[epoch]:
        print(f'{name[1:3]}, ', end='')
    print('')

35 devices: ['D01_Samsung_GalaxyS3Mini', 'D02_Apple_iPhone4s', 'D03_Huawei_P9', 'D04_LG_D290', 'D05_Apple_iPhone5c', 'D06_Apple_iPhone6', 'D07_Lenovo_P70A', 'D08_Samsung_GalaxyTab3', 'D09_Apple_iPhone4', 'D10_Apple_iPhone4s', 'D11_Samsung_GalaxyS3', 'D12_Sony_XperiaZ1Compact', 'D13_Apple_iPad2', 'D14_Apple_iPhone5c', 'D15_Apple_iPhone6', 'D16_Huawei_P9Lite', 'D17_Microsoft_Lumia640LTE', 'D18_Apple_iPhone5c', 'D19_Apple_iPhone6Plus', 'D20_Apple_iPadMini', 'D21_Wiko_Ridge4G', 'D22_Samsung_GalaxyTrendPlus', 'D23_Asus_Zenfone2Laser', 'D24_Xiaomi_RedmiNote3', 'D25_OnePlus_A3000', 'D26_Samsung_GalaxyS3Mini', 'D27_Samsung_GalaxyS5', 'D28_Huawei_P8', 'D29_Apple_iPhone5', 'D30_Huawei_Honor5c', 'D31_Samsung_GalaxyS4Mini', 'D32_OnePlus_A3003', 'D33_Huawei_Ascend', 'D34_Apple_iPhone5', 'D35_Samsung_GalaxyTabA']
Training set 1 devices: 25, 15, 19, 10, 07, 35, 01, 21, 29, 30, 04, 20, 34, 03, 16, 23, 31, 14, 
Testing set 1 devices: 18, 13, 28, 26, 02, 32, 09, 24, 22, 05, 27, 06, 17, 33, 11, 08, 12, 


### Define network parameters and functions

In [5]:
VALIDATION_PERCENTAGE = 12.5 # 1/8
TEST_PERCENTAGE = 40

# define loss function
compute_loss = torch.nn.BCELoss()

# instantiate the model
# net = H4vdmNet()
net = torch.load('models/2024-03-10_02:17_h4vdm.pth')
net = net.to(device)

def compute_similarity(gop1_features, gop2_features):
    diff = torch.subtract(gop1_features, gop2_features)
    norm = torch.norm(diff, 2)
    tanh = torch.tanh(norm)
    return (torch.ones(tanh.shape) - tanh)

def validate_one_step(model, gop1, gop2, label):
    gop1_features = model(gop1, debug=False, device=device)
    gop2_features = model(gop2, debug=False, device=device)
    gop1_features = gop1_features.to(device)
    gop2_features = gop2_features.to(device)

    similarity = compute_similarity(gop1_features, gop2_features).double()
    similarity.to(device)
    
    label = torch.tensor(label, dtype=float, requires_grad=False, device=device)
    label = label.double()
    
    loss = compute_loss(similarity, label)

    return loss, label, similarity

def validate_one_epoch(model, devices):
    model.eval()
    labels = []
    similarities = []
    for i, testing_set_devices in enumerate(devices):
        print(f'Loading validation set {i+1}/{len(devices)}')
        testing_set = GopPairDataset(vision_gop_dataset, N_GOPS_FROM_SAME_DEVICE, N_GOPS_FROM_DIFFERENT_DEVICE, consider_devices=testing_set_devices, shuffle=True)
        testing_set.pair_dataset = testing_set.pair_dataset[:floor(len(testing_set)*(1-TEST_PERCENTAGE/100))] # reduce size by removing TEST_PERCENTAGE % of the dataset
        validation_set = copy.deepcopy(testing_set)
        validation_set.pair_dataset = validation_set.pair_dataset[:floor(len(testing_set)*(1-VALIDATION_PERCENTAGE/100))] # VALIDATION_PERCENTAGE % of the training set is used for validation, works because the dataset is shuffled
        testing_set.pair_dataset = testing_set.pair_dataset[floor(len(testing_set)*(1-VALIDATION_PERCENTAGE/100)):]
        
        print(f'Validating batch {i+1}/{len(devices)}')
        for j in range(len(testing_set)):
            gop1, gop2, label = testing_set[j]
            loss, label, similarity = validate_one_step(model, gop1, gop2, label)
            labels.append(label.item())
            similarities.append(similarity.item())

    return labels, similarities

def compute_ROC(scores, labels, show: bool = True):
    # compute ROC
    fpr, tpr, thresholds = roc_curve(np.asarray(labels), np.asarray(scores), drop_intermediate=False)
    # compute AUC
    roc_auc = auc(fpr, tpr)

    tnr = 1 - fpr
    max_index = np.argmax(tpr + tnr)

    threshold = thresholds[max_index]
    chosen_tpr = tpr[max_index]
    chosen_fpr = fpr[max_index]


    if show is True:
        lw = 2
        plt.figure()
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC (AUC = %0.2f)' % (roc_auc))
        plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
        plt.plot(chosen_fpr, chosen_tpr, 'o', markersize=10, alpha=0.5, label="Threshold = %0.2f" % threshold)
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc="lower right")
        plt.show()
    return threshold, chosen_tpr, chosen_fpr, roc_auc

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(scores, labels, threshold):
    thresholded_scores = scores > threshold

    precision = precision_score(labels, thresholded_scores, average='macro')
    recall = recall_score(labels, thresholded_scores, average='macro')
    f1 = f1_score(labels, thresholded_scores, average='macro')
    accuracy = accuracy_score(labels, thresholded_scores)

    return precision, recall, f1, accuracy

### Perform evaluation

In [6]:
import random
random.seed(10)

labels, similarities = validate_one_epoch(net, testing_set_devices)

Loading validation set 1/7
Testing set has 459 GOP pairs
Validation set has 3213 GOP pairs
Validating batch 1/7


Compute metrics

In [7]:
threshold, tpr, fpr, roc_auc = compute_ROC(similarities, labels)
precision, recall, f1, accuracy = compute_metrics(similarities, labels, threshold)

print(f'Precision: {precision:.2%}')
print(f'Recall: {recall:.2%}')
print(f'F1: {f1:.2%}')
print(f'Accuracy: {accuracy:.2%}')

ValueError: y_true takes value in {} and pos_label is not specified: either make y_true take value in {0, 1} or {-1, 1} or pass pos_label explicitly.